In [1]:
# Imports
import pandas as pd
import numpy as np
import datetime
from datetime import time as dtime
from datetime import timedelta
from datetime import date
import matplotlib as plt
from itertools import product
import openpyxl

# importing historical data for backtest
def get1minBars(csvName):
    
    priceData1minBars_df = pd.read_csv(csvName, usecols=['date', 'open', 'high', 'low', 'close', 'volume'])
    
    # Parse datetime column and divide in 2 columns
    # SPY_Data['datetime'] = pd.to_datetime(SPY_Data['date'].str.strip(), format='%Y%m%d %H:%M:%S')
    priceData1minBars_df.rename(columns={
        'date': 'datetime'
    }, inplace=True)
    
    priceData1minBars_df['datetime'] = pd.to_datetime(priceData1minBars_df['datetime'])
    
    priceData1minBars_df['Date'] = priceData1minBars_df['datetime'].dt.date
    priceData1minBars_df['Time'] = priceData1minBars_df['datetime'].dt.time
    
    # sorting the data by the time column to make sure it is consistent
    priceData1minBars_df.sort_values('datetime', inplace=True)
    
    # Drop original 'date' column
    # SPY_Data.drop(columns=['date'], inplace=True)
    
    # Filter for regular trading hours using datetime column
    priceData1minBars_df = priceData1minBars_df[priceData1minBars_df['datetime'].dt.time >= pd.to_datetime("09:30").time()]
    priceData1minBars_df = priceData1minBars_df[priceData1minBars_df['datetime'].dt.time <= pd.to_datetime("16:00").time()]
    
    # Making sure the order of the columns stays the same 
    priceData1minBars_df = priceData1minBars_df[['Date', 'Time', 'open', 'high', 'low', 'close', 'volume']]
    
    return priceData1minBars_df
    
    


In [2]:
SPY_Data = get1minBars('SPY_1min_data.csv')
SPY_Data = SPY_Data[472980:483000]
SPY_Data

,Date,Time,open,high,low,close,volume
435930,2024-11-01,09:30:00,571.32,571.45,570.70,571.38,6352
435931,2024-11-01,09:31:00,571.40,571.77,571.24,571.66,1660
435932,2024-11-01,09:32:00,571.63,571.82,571.55,571.75,897
435933,2024-11-01,09:33:00,571.79,571.91,571.63,571.88,1316
435934,2024-11-01,09:34:00,571.90,572.33,571.84,572.16,1848
...,...,...,...,...,...,...,...
445945,2024-12-10,10:25:00,605.38,605.38,605.21,605.26,244
445946,2024-12-10,10:26:00,605.25,605.37,605.25,605.28,253
445947,2024-12-10,10:27:00,605.28,605.38,605.27,605.36,206
445948,2024-12-10,10:28:00,605.37,605.44,605.26,605.28,145


In [8]:
def find_candle_streaks(df, min_streak=4):
    """
    Detects streaks of consecutive candles going in the same direction.
    Adds a column 'streak_dir' (+1 for green, -1 for red) and 'streak_len'.

    Parameters
    ----------
    df : DataFrame with 'open' and 'close' columns.
    min_streak : int, minimum streak length to flag.

    Returns
    -------
    streaks_df : subset of df where a streak of length >= min_streak ends.
    """
    df = df.copy()

    # Direction of each candle (+1 = green, -1 = red, 0 = doji)
    df['dir'] = np.sign(df['close'] - df['open'])
    df['dir'].replace(0, np.nan, inplace=True)  # ignore dojis

    # Compute consecutive direction streak lengths
    streak_len = []
    curr_len = 1
    for i in range(1, len(df)):
        if df['dir'].iloc[i] == df['dir'].iloc[i-1]:
            curr_len += 1
        else:
            curr_len = 1
        streak_len.append(curr_len)
    streak_len.insert(0, 1)  # first bar

    df['streak_len'] = streak_len
    df['streak_dir'] = df['dir']

    # Keep only where streak just reached or exceeded threshold
    streaks_df = df[df['streak_len'] >= min_streak].copy()

    return streaks_df

streaks = find_candle_streaks(SPY_Data, min_streak=6)
streaks

,Date,Time,open,high,low,close,volume,dir,streak_len,streak_dir
436035,2024-11-01,11:15:00,574.94,575.21,574.89,575.12,816,1.0,6,1.0
436071,2024-11-01,11:51:00,573.94,573.94,573.46,573.46,408,-1.0,6,-1.0
436174,2024-11-01,13:34:00,572.56,572.60,572.27,572.30,556,-1.0,6,-1.0
436175,2024-11-01,13:35:00,572.29,572.50,572.13,572.20,809,-1.0,7,-1.0
436176,2024-11-01,13:36:00,572.17,572.21,571.85,571.90,794,-1.0,8,-1.0
...,...,...,...,...,...,...,...,...,...,...
445776,2024-12-09,14:06:00,605.46,605.52,605.34,605.42,293,-1.0,7,-1.0
445812,2024-12-09,14:42:00,604.87,604.90,604.66,604.78,664,-1.0,6,-1.0
445906,2024-12-10,09:46:00,605.09,605.26,605.02,605.15,577,1.0,6,1.0
445907,2024-12-10,09:47:00,605.15,605.27,605.07,605.17,526,1.0,7,1.0


In [15]:
def add_ema10_atr10(df, atr_per_session=True):
    """
    Adds:
      - 'ema10' on close (continuous)
      - 'atr10' (Wilder's ATR). By default, resets each session (per Date).

    Expects columns: ['Date','Time','open','high','low','close','volume'].
    Returns the same df with two new columns.
    """
    out = df.copy()
    out[['open','high','low','close']] = out[['open','high','low','close']].astype(float)

    # --- EMA(10) on close (continuous) ---
    out['ema10'] = out['close'].ewm(span=10, adjust=False).mean()

    # --- ATR(10) (Wilder) ---
    if atr_per_session:
        # prev close within the same Date/session
        prev_close = out.groupby('Date')['close'].shift(1)
    else:
        prev_close = out['close'].shift(1)

    # True Range
    tr1 = (out['high'] - out['low']).abs()
    tr2 = (out['high'] - prev_close).abs()
    tr3 = (out['low']  - prev_close).abs()
    tr = np.maximum(tr1, np.maximum(tr2, tr3))
    # first bar of a session: prev_close is NaN → fall back to high-low
    tr = tr.fillna(tr1)

    if atr_per_session:
        out['atr10'] = (
            tr.groupby(out['Date'])
              .transform(lambda s: s.ewm(alpha=1/10, adjust=False).mean())
        )
    else:
        out['atr10'] = tr.ewm(alpha=1/10, adjust=False).mean()

    return out


def mark_pullback_candles(df):
    """
    Marks:
      - green_pullback: green candle after red, with lower high & lower low than prior red
      - red_pullback: red candle after green, with higher high & higher low than prior green
    Returns df with 2 new boolean columns.
    """

    df = df.copy()

    # determine color of each candle
    df['is_green'] = df['close'] > df['open']
    df['is_red']   = df['close'] < df['open']

    # shift previous candle values
    prev_close = df['close'].shift(1)
    prev_open  = df['open'].shift(1)
    prev_high  = df['high'].shift(1)
    prev_low   = df['low'].shift(1)

    prev_is_green = prev_close > prev_open
    prev_is_red   = prev_close < prev_open

    # condition 1: green candle after red
    cond1 = prev_is_red
    cond2 = (df['low'] < prev_low) & (df['high'] < prev_high)
    cond3 = df['is_green']
    df['green_pullback'] = cond1 & cond2 & cond3

    # condition 2: red candle after green
    cond1r = prev_is_green
    cond2r = (df['low'] > prev_low) & (df['high'] > prev_high)
    cond3r = df['is_red']
    df['red_pullback'] = cond1r & cond2r & cond3r
    
        
    return df

def add_entry_signals(df):
    """
    Marks entries when:
      - LONG signal: green_pullback & close > ema10
      - SHORT signal: red_pullback  & close < ema10
    Then schedules the entry at the *next* bar's open (per Date/session).
    Adds columns:
      - signal: {+1 long, -1 short, 0 none} at the signal bar
      - entryNextOpen: True on the bar where you would enter (next bar)
      - side: {+1, -1, 0} aligned with entryNextOpen
      - entry_price: open price on the entry bar (NaN otherwise)
    """
    df = df.copy()

    # vectorized signal at the *current* bar
    cond_long  = df['green_pullback'] & (df['close'] > df['ema10'])
    cond_short = df['red_pullback']   & (df['close'] < df['ema10'])
    df['signal'] = np.select([cond_long, cond_short], [1, -1], default=0)

    # shift the signal so the entry is at the next bar's open, per session (Date)
    df['side'] = df.groupby('Date')['signal'].shift(1).fillna(0).astype(int)

    # entry flags & entry price on the bar where you'd actually execute
    df['entryNextOpen'] = df['side'] != 0
    df['entry_price']   = np.where(df['entryNextOpen'], df['open'], np.nan)
    
    return df


def add_targets_and_stops(df, atr_mult=0.5, use_prior_atr=True):
    """
    On the bar where entry occurs (entryNextOpen=True):
      - Long:  target = entry_price + atr_used * atr_mult
      - Short: target = entry_price - atr_used * atr_mult
    By default, atr_used is the ATR from the *previous* bar to prevent look-ahead.
    Also adds a symmetric stop for convenience.
    """
    out = df.copy()

    # ATR to use on the entry bar
    if use_prior_atr:
        # use ATR from the prior bar within the same session
        atr_used = out.groupby('Date')['atr10'].shift(1)
    else:
        # use ATR from the entry bar itself (can introduce slight look-ahead if ATR updates with the bar)
        atr_used = out['atr10']

    # initialize
    out['targetPrice'] = np.nan
    out['stopPrice']   = np.nan

    long_mask  = (out['entryNextOpen']) & (out['side'] == 1)
    short_mask = (out['entryNextOpen']) & (out['side'] == -1)

    out.loc[long_mask,  'targetPrice'] = out.loc[long_mask,  'entry_price'] + atr_used.loc[long_mask]  * atr_mult
    out.loc[short_mask, 'targetPrice'] = out.loc[short_mask, 'entry_price'] - atr_used.loc[short_mask] * atr_mult

    # optional: symmetric stop
    out.loc[long_mask,  'stopPrice']   = out.loc[long_mask,  'entry_price'] - atr_used.loc[long_mask]  * atr_mult
    out.loc[short_mask, 'stopPrice']   = out.loc[short_mask, 'entry_price'] + atr_used.loc[short_mask] * atr_mult

    return out


scalpSPY = add_ema10_atr10(SPY_Data, atr_per_session=True)
scalpSPY = mark_pullback_candles(scalpSPY)
scalpSPY = add_entry_signals(scalpSPY)
scalpSPY = add_targets_and_stops(scalpSPY)
scalpSPY

,Date,Time,open,high,low,close,volume,ema10,atr10,is_green,is_red,green_pullback,red_pullback,signal,side,entryNextOpen,entry_price,targetPrice,stopPrice
435930,2024-11-01,09:30:00,571.32,571.45,570.70,571.38,6352,571.380000,0.750000,True,False,False,False,0,0,False,NaN,NaN,NaN
435931,2024-11-01,09:31:00,571.40,571.77,571.24,571.66,1660,571.430909,0.728000,True,False,False,False,0,0,False,NaN,NaN,NaN
435932,2024-11-01,09:32:00,571.63,571.82,571.55,571.75,897,571.488926,0.682200,True,False,False,False,0,0,False,NaN,NaN,NaN
435933,2024-11-01,09:33:00,571.79,571.91,571.63,571.88,1316,571.560030,0.641980,True,False,False,False,0,0,False,NaN,NaN,NaN
435934,2024-11-01,09:34:00,571.90,572.33,571.84,572.16,1848,571.669115,0.626782,True,False,False,False,0,0,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445945,2024-12-10,10:25:00,605.38,605.38,605.21,605.26,244,605.303261,0.146114,False,True,False,False,0,0,False,NaN,NaN,NaN
445946,2024-12-10,10:26:00,605.25,605.37,605.25,605.28,253,605.299032,0.143503,True,False,False,False,0,0,False,NaN,NaN,NaN
445947,2024-12-10,10:27:00,605.28,605.38,605.27,605.36,206,605.310117,0.140152,True,False,False,False,0,0,False,NaN,NaN,NaN
445948,2024-12-10,10:28:00,605.37,605.44,605.26,605.28,145,605.304641,0.144137,False,True,False,False,0,0,False,NaN,NaN,NaN
